In [ ]:
# TensorFlow ve Keras temel modülleri
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input,
    Dense,
    Embedding,
    MultiHeadAttention,
    LayerNormalization,
    Dropout
)

# Eğitim için
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Veri hazırlama
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Yardımcı kütüphaneler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


# 📘 TRANSFORMER MİMARİSİNE GİRİŞ – KONU BAŞLIKLARI VE AÇIKLAMALARI

## 1. 🔍 Transformer Nedir?
* LSTM/RNN gibi sıralı yapılara alternatif olarak geliştirilmiştir.

* 2017’de “Attention is All You Need” makalesiyle tanıtılmıştır.

* Doğal dil işleme (NLP), görüntü işleme, zaman serisi ve çok daha fazlasında kullanılabilir.

## 2. 🧠 Self-Attention Mekanizması
Her kelime, diğer kelimelerle olan ilişkisini değerlendirir.

* Query, Key, Value matrisleri üzerinden hesaplanır.

Uzun dizilerdeki bağlam ilişkilerini çok daha etkili şekilde öğrenir.



## 3. 🧩 Multi-Head Attention
* Self-Attention işlemi birden fazla “baş” ile yapılır.

* Farklı temsillerin paralel olarak öğrenilmesini sağlar.

* Her baş, farklı bir bilgiye odaklanabilir (örneğin dilbilgisi, bağlam, kelime ilişkisi).

## 4. 🌐 Positional Encoding
* Transformer'lar sıralı bilgiye doğrudan sahip değildir.

* Pozisyon bilgisini sinüsoidal fonksiyonlarla modele enjekte eder.

* Her kelimenin cümle içindeki konumu böylece anlaşılır hale gelir.

## 5. 🧱 Transformer Bloklarının Yapısı
Her Encoder/Decoder bloğu şu bileşenleri içerir:

* Multi-Head Attention

* Add & Layer Normalization

* Feedforward Neural Network

* Add & Layer Normalization

Her blokta residual bağlantılar vardır (skip connections), eğitim sürecini stabil tutar.

## 6. 🔁 Encoder ve Decoder Yapısı
Encoder:

* Giriş cümlesini işler, gizli temsiller üretir.

Decoder:

* Bu temsillerden yola çıkarak çıkış dizisi üretir.

Masked Attention ile gelecekteki kelimelere bakmayı engeller (örneğin çeviride).

## 7. ⚙️ Transformer’ın Avantajları
* Paralel işlemeye uygun (GPU/TPU için hızlı).

* Uzun bağımlılıkları daha iyi öğrenir.

* Daha az bellekle daha fazla bağlam yakalayabilir.

## 8. 🛠️ Transformer ile Uygulama Senaryoları
* Makine çevirisi (EN → TR)

* Chatbotlar (GPT mimarisi)

* Metin sınıflandırma (BERT gibi)

* Metin tamamlama / üretim (GPT-2, GPT-3)

* Zaman serisi tahmini, görüntü işlemede ViT

-------

# SEQ2SEQ ile TRANSFORMER arasındaki farklar nelerdir ?

### 💡 1. Veri İşleme Yöntemi (Sıralı mı? Paralel mi?)
* LSTM tabanlı modeller veriyi sırayla işler. Her adımda bir kelime alınır, önceki gizli durum aktarılır. Bu nedenle hesaplama zaman bağımlı ve sıralıdır.

* Transformer ise tüm kelimeleri aynı anda işler. Çünkü self-attention tüm diziyi bir kerede görebilir. Bu sayede eğitim süreci çok daha hızlıdır ve paralel hesaplama yapılabilir.


### 🧠 2. Uzun Bağlamları Öğrenme Yeteneği
* LSTM’ler uzun cümlelerde erken gelen kelimeleri unutabilir. Evet, bidirectional LSTM ve attention eklenerek bu kısmen iyileştirilir ama tam çözüm değildir.

* Transformer, her kelimenin diğer tüm kelimelerle olan ilişkisini self-attention ile doğrudan öğrenir. Bu sayede uzun bağlamlar çok daha sağlam öğrenilir.



### 🧲 3. Attention Kullanımı
* LSTM tabanlı modellerde Attention genellikle Encoder’ın son gizli durumuna göre çalışır ve Decoder'da dıştan eklenir.

* Transformer'da ise Attention modelin merkezi parçasıdır. Hem Encoder hem Decoder çok katmanlı Attention bloklarından oluşur. Yani attention kenarda bir eklenti değil, doğrudan yapının kendisidir.

### 🧱 4. Yapısal Fark (Layer Bazında)
* LSTM modeller katman katman ilerlese de her katman kendi içinde sıralı bilgi taşır. Layer’lar arasında veri genellikle bir hidden state olarak geçer.

Transformer’da her katman:

* Multi-Head Attention

* Residual Connection

* Layer Normalization

* Feedforward katman içerir.

Bu yapı sayesinde çok daha derin modeller stabil şekilde eğitilebilir.

### ✅ Özetle Neden Transformer?
* LSTM'ler sıralı ve yavaş, Transformer paralel ve hızlı.

* Transformer, daha uzun bağlamları etkili biçimde öğrenebilir.

* Attention, LSTM'de sonradan eklenir ama Transformer'da yapının kalbidir.

* Eğitimde verimli, ölçeklenebilir ve daha derin mimarilere uygundur.



---------

# 🧾  Frame Tabanlı Transformer Sohbet Modeli
### 📌 Giriş
* Bu çalışmada, Transformer Encoder-Decoder mimarisi kullanılarak geliştirilen frame tabanlı bir sohbet modeli eğitilecektir. Bu model, kullanıcıdan gelen belirli kalıplardaki (frame) girdilere karşılık olarak önceden tanımlanmış çıktıları öğrenmeyi amaçlar.

### 🔍 Amaç
* Bu modelin amacı, çok büyük dil modelleri (örneğin GPT-2, GPT-3) gibi genel amaçlı bir dil zekası üretmek değil; sadece eğitim sırasında verilen input-output çiftleri üzerinden öğrenerek, sınırlı bir etkileşim sağlamaktır.

### 🧠 Model Özellikleri
* Mimari: Transformer Encoder–Decoder (Attention temelli)

* Eğitim verisi: Girdi–çıktı çiftlerinden oluşan küçük bir özel veri kümesi

* Çalışma mantığı: Kullanıcıdan gelen belirli cümleleri tanıyıp, karşılık gelen çıktıyı üretmek

* Genelleme kabiliyeti: Düşüktür, sadece örneklerine benzer girdilere anlamlı cevap verir

* Avantajı: Küçük veriyle çalışabilir, hızlı eğitilir

* Dezavantajı: Ezberci davranır; eğitim dışındaki örüntülerde başarısı düşer

### 📚 Uygulama Alanı
* Menü tabanlı chatbot

* Sık sorulan sorular yanıtlayıcısı (FAQ bot)

* Kapalı alan asistanları (örneğin: otel resepsiyonu, otomatik müşteri temsilcisi)

--------

### Şimdi sizinle frame tabanlı Transformer mantığını inceyeleyelim.Bu adımda kendi oluşturduğum verisetini kullanacağız.Adımları tek tek açıklayarak devam edeceğiz.

-------

# -- Frame Tabanlı Transformerlar -- 

* BU NOTEBOOK ANLATIMINDA RNN SEQ2SEQ MODELLERİN , RNN'LERİN , ANN'LERİN , CNN'LERİN BİLİNDİĞİ VAR SAYILMIŞTIR.ANLATIMLAR DİĞER REPOLARIN DEVAMI OLARAK SUNULACAKTIR.

### 🧩 Frame Tabanlı Transformer Modeli – Adım Adım Plan

2️⃣ 📊 Veriyi pandas ile oku (pd.read_csv)

3️⃣ 📝 Cümleleri listeye al (input ve output ayrı ayrı)

4️⃣ 🔤 Tokenizer oluştur (hem input hem output için)

5️⃣ 📏 Cümleleri tokenize et, pad uygula (maksimum uzunluk belirle)

6️⃣ 📦 Veriyi ayır → X, y_input, y_target (encoder/decoder ayrımı)

7️⃣ 🏗️ Transformer modelini kur (Encoder–Decoder yapısı)

8️⃣ ⚙️ Loss ve optimizer tanımla (örneğin SparseCategoricalCrossentropy)

9️⃣ 🎯 Modeli eğit (model.fit)

🔟 💬 Tahmin fonksiyonu yaz (input ver, output üret)

1️⃣1️⃣ 🧪 Modeli test et (örnek girdilerle dene)

1️⃣2️⃣ 💾 Modeli kaydet (isteğe bağlı olarak model.save())

-------

##  📊 Veriyi pandas ile oku (pd.read_csv)

In [22]:
import pandas as pd

df = pd.read_csv(r"C:\Users\hdgn5\OneDrive\Masaüstü\Transformerlar\Konu Anlatımları\Tenserflow Transformer - Teorik\örnek_set.csv")
print(df.head())


                 input                                             output
0              Merhaba           Merhaba, size nasıl yardımcı olabilirim?
1            Nasılsın?           İyiyim, teşekkür ederim. Siz nasılsınız?
2             Adın ne?  Ben bir yapay zekâ asistanıyım. Adım yok ama y...
3      Kaç yaşındasın?                        Benim yaşım yok, dijitalim!
4  Bugün günlerden ne?  Maalesef tarih bilgim yok, ama sistem saatinde...


----

## 3️⃣ 📝 Cümleleri listeye al (input ve output ayrı ayrı)

In [23]:
input_texts = df['input'].astype(str).to_list()
target_texts = df['output'].astype(str).to_list()

-----

## 4️⃣ 🔤 Tokenizer oluştur (hem input hem output için)

* Aslında seq2seq modellerde kullandığımızdan çok daha farklı bir işlem yapmayacağız.Neredeyse aynı bile denebilir.Üstüne daha iyi hale getirmek için eklentiler açacağım.

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# START VE END TOKENLARINI CÜMLEYE EKLEDİK
target_texts_with_token = ['<start> ' + t + ' <end>' for t in target_texts]

# TOKENIZERLARI FİLTRE VE OOV TOKEN İLE OLUŞTURDUK
input_tokenizer = Tokenizer(filters="",oov_token="<OOV>")
target_tokenizer = Tokenizer(filters="",oov_token="<OOV>")

# TOKENLARI FİT ETTİK
input_tokenizer.fit_on_texts(input_texts)
target_tokenizer.fit_on_texts(target_texts_with_token)

# CÜMLELERİ TOKENİZE EDECEĞİZ
input_sequences = input_tokenizer.texts_to_sequences(input_texts)

decoder_input_sequences = target_tokenizer.texts_to_sequences(['<start> ' + t for t in target_texts])

decoder_target_sequences = target_tokenizer.texts_to_sequences([t+' <end>' for t in target_texts])

# MAX UZUNLUĞU HESAPLADIK
max_input_len = max(len(seq) for seq in input_sequences)
max_output_len = max(len(seq) for seq in input_sequences)

max_len = max(max_input_len,max_output_len)

# PAD İŞLEMİNİ GERÇEKLEŞTİRDİK
encoder_input = pad_sequences(input_sequences,maxlen=max_len,padding="post")
decoder_input = pad_sequences(decoder_input_sequences,maxlen=max_len,padding="post")
decoder_target = pad_sequences(decoder_target_sequences,maxlen=max_len,padding="post")  

# VOCAB_SİZE BELİRLEDİK.
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

In [25]:
print(input_vocab_size)
print(target_vocab_size)

104
249


### Bu işlemleri daha iyi bir hale getirebiliriz.Mesela verileri temizleriz ya da özel tokenları elle ekleriz.Gelin bu halden daha iyi bir hale getirelim bu tokenizerları.

In [26]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 🔤 1. Özel tokenları tanımla
SPECIAL_TOKENS = ['<pad>', '<start>', '<end>', '<OOV>']

### 🧹 2. Temizlik fonksiyonu
def clean_text(text):
    text = text.lower().strip()
    text = text.replace("’", "'").replace("“", '"').replace("”", '"')
    text = text.replace("–", "-").replace("...", ".")
    return text

### 📝 3. Cümleleri temizle
input_texts_clean = [clean_text(t) for t in input_texts]
target_texts_clean = [clean_text(t) for t in target_texts]

### 🪄 4. Target cümlelere start ve end token ekle
target_texts_with_tokens = ['<start> ' + t + ' <end>' for t in target_texts_clean]

### 🧠 5. Tokenizer oluştur ve özel tokenları vocab'a enjekte et
def build_tokenizer(texts, num_words=None):
    tokenizer = Tokenizer(filters='', oov_token='<OOV>', num_words=num_words)
    tokenizer.fit_on_texts(SPECIAL_TOKENS + texts)  # özel tokenları ilk sıraya ekle
    return tokenizer

input_tokenizer = build_tokenizer(input_texts_clean)
target_tokenizer = build_tokenizer(target_texts_with_tokens)

### 🔢 6. Cümleleri tokenize et
input_sequences = input_tokenizer.texts_to_sequences(input_texts_clean)
decoder_input_sequences = target_tokenizer.texts_to_sequences(
    ['<start> ' + t for t in target_texts_clean]
)
decoder_target_sequences = target_tokenizer.texts_to_sequences(
    [t + ' <end>' for t in target_texts_clean]
)

### 📏 7. Maksimum uzunluk belirle (tek max_len kullanılacak)
max_input_len = max(len(seq) for seq in input_sequences)
max_target_len = max(len(seq) for seq in decoder_target_sequences)
max_len = max(max_input_len, max_target_len)

### 🧱 8. Padding uygula
encoder_input = pad_sequences(input_sequences, maxlen=max_len, padding='post')
decoder_input = pad_sequences(decoder_input_sequences, maxlen=max_len, padding='post')
decoder_target = pad_sequences(decoder_target_sequences, maxlen=max_len, padding='post')

### 📌 10. Vocab boyutları
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1


-----

## 📦 6️⃣ Veriyi ayır → X, y_input, y_target

* Bu adımda modelin 3 temel girdisini oluşturacağız:


X =>	Encoder'a gidecek input (kullanıcının mesajı)

y_input =>	Decoder'a giriş olarak verilen diziler (<start token'lı)

y_target =>	 Modelin üretmesi gereken doğru çıktı (<end token'lı)

#### 🧠 Kısaca:
y_input = Decoder’a “tahmin etmeye başla” demek

y_target = “Ne tahmin etmen gerekiyordu” demek

Bu ikisinin ayrılması = Teacher Forcing dediğimiz tekniğin temelidir

#### 🛠️ Eğer bu ayrımı yapmazsan...
Model decoder’a giriş alamaz, çıktı üretemez

loss hesaplaması yanlış olur (çıktıyı neyle kıyaslayacağını bilemez)

Eğitim başarısız olur ya da saçma sonuçlar üretir

#### 🎓 Sonuç:
Encoder–Decoder mimarilerde doğru input ve target ayrımı, modelin dili öğrenmesinin temel taşıdır.

Bu nedenle bu adım, "Transformer’ın konuşmayı öğrenmesini sağlayan yapı taşlarının montajı" gibidir diyebiliriz 🧩

In [27]:
# Encoder input → kullanıcı cümlesi
X = encoder_input

# Decoder input → <start> ile başlayan hedef cümle
y_input = decoder_input

# Decoder target → <end> ile biten gerçek cevap
y_target = decoder_target


In [28]:
print("Encoder input örneği:", X[0])
print("Decoder input örneği:", y_input[0])
print("Decoder target örneği:", y_target[0])

Encoder input örneği: [24  0  0  0  0  0  0  0  0  0  0  0]
Decoder input örneği: [ 2 54 14 55  7 56  0  0  0  0  0  0]
Decoder target örneği: [54 14 55  7 56  3  0  0  0  0  0  0]


----

## 🧱 7️⃣ Transformer Encoder–Decoder Modeli (TensorFlow/Keras)


* Model içinde çok daha farklı terimler göreceğiz.Hatırlatmak için tekrar yazıyorum.

### 📍 Positional Encoding Nedir?
* Transformer modelleri, RNN'lerin aksine veriyi sırayla işlemez — yani kelimelerin hangi sırada geldiğini doğrudan bilmez.

Çünkü Attention mekanizması aynı anda tüm kelimelere bakar.

Bu da sırayı "unutmasına" sebep olur.

* İşte bu yüzden Positional Encoding devreye girer:

####  🧠 Tanım:
* Positional Encoding, her kelimenin sırasını modele bildirmek için embedding’lere eklenen özel bir sinyaldir.

#### 🔢 Nasıl çalışır?
* Her pozisyon (0, 1, 2, ...) için bir vektör üretilir.

* Bu vektörler sinüs ve kosinüs fonksiyonlarıyla oluşturulur.

* Böylece model şu farkı anlayabilir:

* “2. kelime ile 5. kelime arasında ne kadar mesafe var?”

#### 📌 Sonuç:
* Positional Encoding, kelimelerin sırasını modele gizlice öğretir

* Embedding’lere eklenir (toplama yapılır)

* Model artık sadece ne dediğini değil, ne zaman dediğini de bilir

#### Aslında bu konu benim de merakımdı.Yani neden sin/cos kullanıyoruz?Bu hangi amaca hitap ediyor ki?Gelin beraber inceleyelim.

### 🧩 Neden sin/kos?
* Sinüs ve kosinüs, ardışık pozisyonlar arasındaki farkı düzgün ve öğrenilebilir biçimde taşır

Daha da önemlisi:

* Model, mutlak pozisyon değil, pozisyonlar arası fark üzerinde öğrenir



### Gelin Positional  Encoder'ın kod bloklarına bakalımm.

In [ ]:
import tensorflow as tf

def positionel_encoding(max_len,d_model):

# ➤ max_len kadar sıradaki pozisyon numaralarını tek tek bir sütun vektörü haline getirdik.
    pos = tf.range(max_len,dtype = tf.float32)[:, tf.newaxis]

# ➤ d_model, her kelimenin embedding (veya temsil) vektörünün boyutudur. Aşağıdaki satır ise bu boyutlar boyunca pozisyonel sinyallerin nasıl yayılacağını belirlemek için kullanılır."
    i = tf.range(d_model , dtype=tf.float32)[tf.newaxis,:]

# ➤ pozisyona göre hangi frekansta sinyal (açı) verileceğini belirler.
    angle_rates =  1 / tf.pow(100000.0 ,(2 * ( i//2)) / tf.cast(d_model,tf.float32))

# ➤ Her pozisyondaki kelime için, her embedding boyutuna karşılık gelen açısal (frekanslı) sinyali hesapla
    angel_rads = pos * angle_rates

    sines = tf.math.sin(angel_rads[:, 0::2])
    cosines = tf.math.cos(angel_rads[:, 1::2])

    pos_encoding = tf.concat([sines,cosines] , axis = 1)

    return pos_encoding[tf.newaxis , ...]

## 🔢 1. pos = tf.range(max_len)[:, tf.newaxis]

* Çıktı olarak

[[0]
 [1]
 [2]
 [3]
 [4]]

 ##### 📌 Anlamı:

* Bu, her pozisyonu temsil eder.
* Yani "Merhaba dünya bugün nasılsın?" cümlesi varsa:

"Merhaba" → pozisyon 0

"dünya" → pozisyon 1

...

* Yani pos = kaçıncı kelime olduğunu belirler.

➤ max_len kadar sıradaki pozisyon numaralarını tek tek bir sütun vektörü haline getirdik.

### 🎯 Kısaca:
* tıpkı tokenizer'da her kelimeye ID verir gibi burada da her pozisyona ID veriyoruz

* Ama ID'ler sayısal değil, birazdan sinyale dönüşecek 🧠📡

## 2️⃣  i = tf.range(d_model)[tf.newaxis, :]

#### * ✅ Ne yapar?

Her embedding boyutunun indeksini çıkarır. Yani "0. boyut, 1. boyut, 2. boyut, ..." gibi değerleri verir.


##### *  Örnek: d_model = 4
[[0, 1, 2, 3]]

0 → embedding'in 1. boyutu

1 → embedding'in 2. boyutu

...

Bu, her vektör boyutu için indeksleri temsil eder

Ve angle_rates hesaplanırken her boyuta farklı sinyal verebilmek için lazım

##### 🧠 Amacı ne?
Her embedding boyutuna ayrı bir sinyal frekansı verebilmek.
Bu, positional encoding'in sinüs/kosinüs hesaplarında kullanılır.

##### 📌 Şunu unutma:
* pos → pozisyonlar (0. kelime, 1. kelime...)

* i → embedding boyutu indeksleri (0. boyut, 1. boyut...)

* pos satırdır → her kelime sırası için bir vektör

* i sütundur → her vektörün kaçıncı boyutu olduğunu belirtir

#### 📌 Kısa ve Net Tanım:
* i = tf.range(d_model)[tf.newaxis, :]

➤ Her pozisyon vektörünün hangi boyutuna hangi frekansla sinyal verileceğini belirlemek için kullanılır.

## 3️⃣ 📐 angle_rates = 1 / tf.pow(10000., (2 * (i // 2)) / d_model)

* Bu satır = pozisyona göre hangi frekansta sinyal (açı) verileceğini belirler.

* Yani: “Bu embedding boyutunda nasıl bir sinyal eğrisi kullanayım?”

#### 🧠 Ne Anlama Geliyor?
🎯 Amaç:

* Her embedding boyutuna farklı frekanslı sinyal vermek

* Böylece pozisyon farkları hem küçük hem büyük ölçeklerde yakalanabilir

### 🔍 Parça Parça İnceleyelim:
✅ i // 2

* sin/cos dönüşümü için çift-tek ayrımı

* Boyutları 2’şer 2’şer gruplandırmak için

✅ (2 * (i // 2)) / d_model

* d_model’e oranla ne kadar hızlı değişim olacak?

* Daha küçük boyutlarda yüksek frekans (detaylı sinyal)

* Daha büyük boyutlarda düşük frekans (genel yapı)

✅ tf.pow(10000., ...)

* Bu, frekansı üstel olarak azaltmak için kullanılır

* 10000 sabiti → paper’da önerilen sabit; sinyalleri yeterince seyreltiyor



####  💡 Neden Bu Kadar Özenli?
* Çünkü bazı kelime ilişkileri kısa mesafededir (örneğin: "merhaba → nasılsın"),
bazıları uzundur ("bugün → dışarısı → hava → güzel")

* Modelin her mesafeye göre sinyali olsun diye çok frekanslı bir yapı gerekir → işte bu satır bunun içindir.

Terim ====== 	Ne işe yarar?

10000 =>	Üssün bazı, frekansları yaymak için

2i/d_model =>	Embedding boyutuna göre frekansı ölçeklemek

i // 2 =	Aynı frekansı sin/cos olarak çift-tek boyutlara dağıtmak

## 4️⃣ 📡 angle_rads = pos * angle_rates

* Her pozisyondaki kelime için, her embedding boyutuna karşılık gelen açısal (frekanslı) sinyali hesaplar.

##### 📐 Ne Oluyor Burada?
* pos → her kelimenin sırası (0, 1, 2, 3, ...) → shape: [max_len, 1]

* angle_rates → her embedding boyutu için frekans katsayısı → shape: [1, d_model]

## 5️⃣ 🎛️ Sin / Cos Uygulama ve Positional Encoding Oluşturma

#### 🔎 Satır Satır Açıklama:
🧪 1. sines = sin(angle_rads[:, 0::2])

* Her kelime için çift numaralı boyutlara sinüs uygula

* Yani: 0, 2, 4, 6, 8, ...

🧊 2. cosines = cos(angle_rads[:, 1::2])

* Her kelime için tek numaralı boyutlara kosinüs uygula

* Yani: 1, 3, 5, 7, 9, ...

🎯 Sin/cos karışımı → faz farkı ekleyerek zenginleştirilmiş sinyal yapısı oluşturur
(sin ve cos aynı frekansta ama 90 derece kayık = sinyal farklarını büyütüyor)

#### 🔗 3. tf.concat([sines, cosines], axis=-1)
* sines ve cosines dizilerini yan yana birleştir

* Sonuç = [max_len, d_model] boyutunda positional sinyal tablosu



#### 🧱 4. return pos_encoding[tf.newaxis, ...]
* Bu satırla shape şu hale gelir: [1, max_len, d_model]

* Yani bu encoding, batch’e uygun hale gelir ve direkt embedding ile toplanabilir:

## 🧩 Positional Encoding – Adım Adım Özet

| Adım                  | Ne yaptı?                                                                 |
|-----------------------|---------------------------------------------------------------------------|
| `pos`                | Her kelimenin cümledeki sırasını aldı                                     |
| `i`                  | Embedding boyutlarının indekslerini oluşturdu                             |
| `angle_rates`        | Her embedding boyutu için frekans hesaplama katsayısı belirledi            |
| `pos * angle_rates`  | Her pozisyon-boyut kombinasyonu için açısal değerler hesapladı             |
| `sin / cos`          | Sinyalleri faz farkıyla dönüştürerek pozisyonel bilgi kazandırdı           |
| `concat`             | Sin ve cos verilerini birleştirerek final sinyal vektörünü oluşturdu       |
| `+ embedding`        | Embedding vektörüne pozisyon bilgisini ekledi → model sıraları “görür” ✅   |


## 📐 Positional Encoding Son Demler

- Positional Encoding, aslında **embedding katmanına eklenen sinyaldir**.
- Model sırayı bilemediği için, bu sinyalle **kelimenin kaçıncı sırada olduğunu** anlar.
- İlk olarak, cümledeki **her pozisyonu (`pos`)** temsil eden bir vektör oluşturduk.
- Ardından, her **embedding boyutu (`d_model`)** için bir boyut vektörü (`i`) oluşturduk.
- Sonra bu boyutlara göre **açısal frekans katsayıları (`angle_rates`)** belirledik.
- Bu frekanslar ile pozisyonları çarparak **açısal değerler (`angle_rads`)** elde ettik.
- Daha sonra çift indeksli boyutlara `sin`, tek indeksli boyutlara `cos` uyguladık.
- Bu `sin` ve `cos` sinyallerini birleştirerek **pozisyonel sinyal vektörü** oluşturduk.
- En sonunda bu vektörü embedding çıktısıyla toplayarak **modelin sıralamayı "görmesini" sağladık**.


-----------
------

# Şimdi Positionel Encodingden Sonra Gelen Embedding Katmanını İnceleyelim

-----
-----

#   ✅ Embedding + Positional Encoding Katmanı (Fonksiyon)

* Aslında embedding katmanını biliyoruz.Metinsel değerlerde kullandığımız bir vektör dönüştürücü.Bundna önce de size zaten positionel encoding i anlatmıştım.Şimdi bu positionel encoding katmaını embedding e bağlamamız gerekecek.Gelin bu işlemlere bakalım.


In [30]:
from tensorflow.keras.layers import Input, Embedding, Add, Lambda

def token_and_position_embedding(vocab_size, max_len, d_model):
    inputs = Input(shape=(None,), name="input_tokens")

    x = Embedding(input_dim=vocab_size, output_dim=d_model)(inputs)

    # positional encoding önceden sabit oluşturulmuş (max_len kadar)
    pos_encoding = positionel_encoding(max_len, d_model)

    # Lambda ile slice işlemini runtime’da yapıyoruz
    def slice_position(x_input):
        seq_len = tf.shape(x_input)[1]
        return pos_encoding[:, :seq_len, :]

    pos_slice = Lambda(slice_position)(x)

    x = Add()([x, pos_slice])

    return Model(inputs=inputs, outputs=x, name="TokenPosEmbedding")


### 📌 Kullanımı:

In [32]:
# encoder_inputs bizim encoder için Input katmanımız
from tensorflow.keras import layers

encoder_inputs = layers.Input(shape=(None,), name="encoder_input")
embedding_layer = token_and_position_embedding(
    vocab_size=input_vocab_size, max_len=max_len, d_model=128
)

# çağırdığımızda sanki bir layer gibi davranır
x = embedding_layer(encoder_inputs)


--------
----




# Şimdi Sırada Encoder Yapısı Var.Merak Etmeyin Bu Fonksiyonları Daha Güçlü Hale Getireceğiz.Bu Yalnızca Bir Başlangıçç...

-----
-----

## 🏗️ build_encoder() Fonksiyonu — Tek Blok Encoder Yapısı

####  🧱 build_encoder() Fonksiyonunun Amacı

- Bu fonksiyon, Transformer mimarisindeki **Encoder bloğunu** oluşturur.
- Encoder, verilen token dizisinden (yani giriş cümlesinden), **sıralı ve bağlama duyarlı bir vektör temsili** üretir.
- İçerisinde:
  - Embedding + Positional Encoding (kelimenin anlamı + sırası)
  - Multi-Head Self Attention (kelimeler arası ilişki öğrenimi)
  - Feed Forward Network (her pozisyonu bireysel işler)
  - Residual bağlantılar + Layer Normalization (öğrenmeyi stabilize eder)
- Modelin giriş kısmında (input side) yer alır ve decoder’a bilgi sağlar.


## 🔧 Feed Forward Network (FFN) Nedir?

- Transformer’daki her encoder/decoder bloğunda bulunan küçük sinir ağıdır.
- Her kelime pozisyonuna **ayrı ayrı** uygulanır.
- Yapısı genellikle:

  Dense(d_model * 4, activation='relu') → Dense(d_model)
  
- Attention çıktısını işler, modeli derinleştirir ve anlam çıkarım gücünü artırır.


#### 🎯 FFN nedir, ne işe yarar?
* Transformer içinde her bir pozisyon (token) için ayrı ayrı uygulanan küçük bir sinir ağıdır.

* Yani: Kelime başına “bireysel olarak düşünme” mekanizmasıdır 🤖🧠

In [ ]:
x = Dense(d_model * 4, activation='relu')(x)
x = Dense(d_model)(x)

Bu şu anlama gelir:

* İlk katman: Genişlet → d_model → 4 × d_model

* ReLU aktivasyonu: doğrusal olmayanlık katar

* İkinci katman: Daralt → 4 × d_model → d_model



In [ ]:
from tensorflow.keras import layers, Model

def build_encoder(vocab_size, max_len, d_model, num_heads):
    # 1️⃣ Giriş katmanı
    encoder_inputs = layers.Input(shape=(None,), name="encoder_input")

    # 2️⃣ Embedding + Positional Encoding (hazır fonksiyon)
    embedding_layer = token_and_position_embedding(vocab_size, max_len, d_model)
    x = embedding_layer(encoder_inputs)

    # 3️⃣ Multi-Head Self-Attention
    attn_out = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)

    # 4️⃣ Residual + Layer Normalization
    x = layers.Add()([x, attn_out])
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # 5️⃣ Feed Forward Network
    ffn = layers.Dense(d_model * 4, activation='relu')(x)
    ffn = layers.Dense(d_model)(ffn)

    # 6️⃣ Residual + Layer Normalization
    x = layers.Add()([x, ffn])
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # 7️⃣ Modeli döndür
    return Model(inputs=encoder_inputs, outputs=x, name="TransformerEncoder")


In [ ]:
# 📌 Kullanımı:

encoder = build_encoder(
    vocab_size=input_vocab_size,
    max_len=max_len,
    d_model=512,
    num_heads=4
)

encoder.summary()

Model: "TransformerEncoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TokenPosEmbedding   │ (None, 12, 512)   │     55,296 │ encoder_input[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 12, 512)   │  4,200,960 │ TokenPosEmbeddin… │
│ (MultiHeadAttentio… │                   │            │ TokenPosEmbeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 12, 512)   │          0 │ TokenPosEmbeddin… │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 12, 512)   │      1,024 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 12, 2048)  │  1,050,624 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 12, 512)   │  1,049,088 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 12, 512)   │          0 │ layer_normalizat… │
│                     │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 12, 512)   │      1,024 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,358,016 (24.25 MB)

 Trainable params: 6,358,016 (24.25 MB)

 Non-trainable params: 0 (0.00 B)

### 🎯 Self-Attention Neydi?
* Bir kelimenin diğer kelimelere olan bağlamını öğrenmesi.

Örnek:

* “Kedi koltuğun üstünde uyuyor.”

* “kedi” → dikkatini “uyuyor” kelimesine vermeli

* “koltuğun” → konum bilgisiyle ilişkili

🧠 Self-attention bunu yapar:

* Her kelime, diğer kelimelerle olan ilişkisini kendisi çözer.

## 🤔 Peki neden Multi-Head Attention?

### 🔍 1. Farklı Bakış Açıları Sağlar

Her “head” = farklı bir öğrenme perspektifi sağlar.

#### 🧠 Örnek:

| Head 1              | Head 2               | Head 3                    |
|---------------------|----------------------|---------------------------|
| dilbilgisel ilişki  | duygusal tonlama     | zamansal bağlam          |
| "özne-fiil" ilişkisi | "sevgi/korku" analizi | "önceki/sonraki kelime" ilişkisi |

Tek bir self-attention bu bağlamları aynı anda yakalayamaz.  
Ama birden fazla attention head, **paralel zihinler gibi** çalışarak her biri farklı bir bilgi yönüne odaklanabilir. 🧠🧠🧠




### 🔍 2. Farklı Projeksiyon Uzaylarında Hesaplama

Her head:
- Kendi `Wq`, `Wk`, `Wv` ağırlıklarını kullanır
- Yani aynı input üzerinden farklı projeksiyonlar oluşturur
- Bu, daha **geniş ilişki çeşitliliği** öğrenmesini sağlar



### 🔍 3. Zenginleştirilmiş Temsil

Tüm head'lerin çıktısı birleştirilir (`concat`) ve ardından bir `Dense` katmanla orijinal boyuta dönülür:


Bu sayede farklı bağlamlar içeren çıktılar, tek bir güçlü vektörde toplanır.


### ✅ Özet

| Özellik                  | Tek Head             | Multi-Head                         |
|--------------------------|----------------------|------------------------------------|
| Perspektif               | Tek bakış açısı      | Farklı bakış açılarının birleşimi  |
| Hesaplama                | Sabit projeksiyon    | Farklı ağırlıklarla farklı dikkat  |
| Öğrenme kapasitesi       | Sınırlı              | Daha esnek ve güçlü                |
| Bağlam çeşitliliği       | Düşük                | Yüksek                             |

Multi-head attention sayesinde model, dilin çok katmanlı doğasını daha iyi kavrayabilir. 🔥


## 🧱 Profesyonel build_encoder() — Temiz ve Katmanlaştırılmış

In [33]:
from tensorflow.keras.layers import Input, Dense, Embedding, Dropout, LayerNormalization, MultiHeadAttention


def add_and_norm(x,sublayer_output,dp=0.3):
    dropped = layers.Dropout(dp)(sublayer_output)
    added = layers.Add()([x,dropped])

    return layers.LayerNormalization(epsilon=1e-6)(added)

def feed_forward_network(d_model):
    return tf.keras.Sequential([
        layers.Dense(d_model * 4, activation='relu'),
        layers.Dense(d_model),
        layers.Dropout(0.3)
    ])


In [34]:
def encoder_block(x, d_model, num_heads, dropout_rate=0.1):
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    x = add_and_norm(x, attn_output, dp=dropout_rate)

    ffn = tf.keras.Sequential([
        layers.Dense(d_model * 4, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(d_model)
    ])
    ffn_output = ffn(x)
    x = add_and_norm(x, ffn_output, dp=dropout_rate)

    return x


In [35]:
def build_encoder(vocab_size, max_len, d_model, num_heads, num_layers=1, dropout_rate=0.1):
    encoder_inputs = layers.Input(shape=(None,), name="encoder_input")

    embedding_layer = token_and_position_embedding(vocab_size, max_len, d_model)
    x = embedding_layer(encoder_inputs)

    for _ in range(num_layers):
        x = encoder_block(x, d_model, num_heads, dropout_rate)

    return Model(inputs=encoder_inputs, outputs=x, name="TransformerEncoder")


In [36]:
encoder = build_encoder(
    vocab_size=input_vocab_size,
    max_len=max_len,
    d_model=256,
    num_heads=4,
    num_layers=4  # ya da kaç katman görmek istiyorsan
)

encoder.summary()


Model: "TransformerEncoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TokenPosEmbedding   │ (1, None, 256)    │     27,648 │ encoder_input[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, None, 256)    │  1,051,904 │ TokenPosEmbeddin… │
│ (MultiHeadAttentio… │                   │            │ TokenPosEmbeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (1, None, 256)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (1, None, 256)    │          0 │ TokenPosEmbeddin… │
│                     │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (1, None, 256)    │        512 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (1, None, 256)    │    525,568 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (1, None, 256)    │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (1, None, 256)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, None, 256)    │        512 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (1, None, 256)    │  1,051,904 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (1, None, 256)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (1, None, 256)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, None, 256)    │        512 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (1, None, 256)    │    525,568 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (1, None, 256)    │          0 │ sequential_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (1, None, 256)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (1, None, 256)    │        512 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 6,341,632 (24.19 MB)

 Trainable params: 6,341,632 (24.19 MB)

 Non-trainable params: 0 (0.00 B)

-----------

## 🔁 Hatırlatma: Transformer Decoder Ne Yapar?
### 📤 Encoder’dan gelen bilgileri kullanır
### 🧠 Kendi geçmiş çıktılarıyla yeni kelime tahmini yapar

#### 🔧 Yapısı:
Her decoder bloğu şunları içerir:

* Masked Multi-Head Self Attention

→ Kendi önceki çıktılara bakar (geleceği görmez ❌)

* Encoder–Decoder Attention

→ Encoder’ın çıktısına dikkat eder

* Feed Forward Network

→ Kelimeyi anlam olarak işler

* Residual + LayerNorm

→ Her adımda stabilite sağlar

### 🔍 1️⃣ Decoder’da Neden Yine Multi-Head Attention?
Çünkü decoder da:

* Kelimeler arasındaki ilişkileri kurmak istiyor

* Ama bunu paralel ve bağlamlı şekilde yapmak istiyor

* Aynı encoder gibi, decoder da:

* Kendi geçmiş çıktılarının bağlamını anlamaya çalışır

* Ve bunu birden fazla “bakış açısıyla” yaparsa çok daha zengin temsil oluşur

### 🎯 Multi-head ne sağlar?

* Head 1: dilbilgisel yapı

* Head 2: geçmiş anahtar kelimeye dikkat

* Head 3: zaman/zaman dışı bağlantı

Hepsi birleştirilir = çok katmanlı anlam çözümü

### 📚 GPT-2, GPT-3, ChatGPT hepsi ne kullanıyor?
* Transformer decoder only

* Hepsi multi-head self-attention kullanıyor

Çünkü bu:

* Paralel

* Dikkatli

* Ölçeklenebilir

Her kelimenin diğerleriyle bağını çoklu seviyede kurabiliyor

### ✅ 1️⃣ decoder_block() Fonksiyonu

* Bunu önceki encoder yapısına göre tanımlıyoruz. 2 Attention + 1 FFN + residual + dropout + layernorm:

In [ ]:
def decoder_block(x,enc_output,d_model,num_heads,dp=0.3):

    attn1 = layers.MultiHeadAttention(num_heads=num_heads,key_dim=d_model)(x,x)

    x = add_and_norm(x,attn1,dp=dp)

    attn2 = layers.MultiHeadAttention(num_heads=num_heads,key_dim=d_model)(x,enc_output)
    x = add_and_norm(x,attn2,dp = dp)

    ffn = tf.keras.Sequential([
        layers.Dense(d_model * 4,activation="relu"),
        layers.Dropout(dp),
        layers.Dense(d_model)
        ])
    
    ffn_out = ffn(x)
    x = add_and_norm(x,ffn_out,dp=dp)

    return x

### ✅ 2️⃣ build_decoder() Fonksiyonu

In [ ]:
def build_decoder(vocab_size, max_len, d_model, num_heads, num_layers=4, dropout_rate=0.1):
    decoder_inputs = layers.Input(shape=(None,), name="decoder_input")
    encoder_outputs = layers.Input(shape=(None, d_model), name="encoder_output")  # DİKKAT!

    embedding_layer = token_and_position_embedding(vocab_size, max_len, d_model)
    x = embedding_layer(decoder_inputs)

    for _ in range(num_layers):
        x = decoder_block(x, encoder_outputs, d_model, num_heads, dropout_rate)

    return Model(inputs=[decoder_inputs, encoder_outputs], outputs=x, name="TransformerDecoder")


#### 📌 Kullanımı:

In [ ]:
decoder = build_decoder(vocab_size=input_vocab_size,
                        max_len=max_len,
                        d_model = 256,
                        num_heads= 6,
                        num_layers= 5)

decoder.summary()

Model: "TransformerDecoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ decoder_input       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TokenPosEmbedding   │ (None, 12, 256)   │     27,648 │ decoder_input[0]… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 12, 256)   │  1,577,728 │ TokenPosEmbeddin… │
│ (MultiHeadAttentio… │                   │            │ TokenPosEmbeddin… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_102         │ (None, 12, 256)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_62 (Add)        │ (None, 12, 256)   │          0 │ TokenPosEmbeddin… │
│                     │                   │            │ dropout_102[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 12, 256)   │        512 │ add_62[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_output      │ (None, None, 256) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 12, 256)   │  1,577,728 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ encoder_output[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_104         │ (None, 12, 256)   │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_63 (Add)        │ (None, 12, 256)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_104[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 12, 256)   │        512 │ add_63[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_17       │ (None, 12, 256)   │    525,568 │ layer_normalizat… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_106         │ (None, 12, 256)   │          0 │ sequential_17[0]… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_64 (Add)        │ (None, 12, 256)   │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_106[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 12, 256)   │        512 │ add_64[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 12, 256)   │  1,577,728 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_108         │ (None, 12, 256)   │          0 │ multi_head_atten

 Total params: 18,440,448 (70.34 MB)

 Trainable params: 18,440,448 (70.34 MB)

 Non-trainable params: 0 (0.00 B)

----------
-------

## 🎯 Encoder + Decoder birleşimiyle tam bir Transformer modeli kuruyoruz.

Bu, eğitimde kullanılacak olan "tam model" olacak.

* Encoder input → Encoder

* Decoder input → Decoder

Son olarak:

* Decoder'ın çıktısı → Dense(vocab_size) → tahmin edilen kelime dizis

### 🧱 1️⃣ build_transformer() Fonksiyonu 👇

In [ ]:
def build_transformer(input_vocab_size,target_vocab_size,max_len,d_model,num_heads,num_layers,dp=0.3):
    
    encoder_inputs = layers.Input(shape=(None,) ,name="encoder_inputs")
    decoder_inputs = layers.Input(shape=(None,) ,name ="decoder_inputs")

    encoder = build_encoder(input_vocab_size,max_len,d_model,num_heads,num_layers,dp)
    decoder = build_decoder(target_vocab_size,max_len,d_model,num_heads,num_layers,dp)

    enc_output = encoder(encoder_inputs)
    dec_output = decoder([decoder_inputs,enc_output])

    final_outputs = layers.Dense(target_vocab_size,activation="softmax" , name="output_layer")(dec_output)

    return Model(inputs=[encoder_inputs,decoder_inputs],outputs =final_outputs , name="TransformerModel")

In [ ]:
transformer = build_transformer(
    input_vocab_size=60,
    target_vocab_size=60,
    max_len=max_len,
    d_model=128,
    num_heads=4,
    num_layers=4
)
transformer.summary()


Model: "TransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TransformerEncoder  │ (1, None, 128)    │  1,591,808 │ encoder_inputs[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TransformerDecoder  │ (1, None, 128)    │  2,648,064 │ decoder_inputs[0… │
│ (Functional)        │                   │            │ TransformerEncod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (1, None, 60)     │      7,740 │ TransformerDecod… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,247,612 (16.20 MB)

 Trainable params: 4,247,612 (16.20 MB)

 Non-trainable params: 0 (0.00 B)

------------

# ⚙️ Loss + optimizer fonksiyonları

In [ ]:
loss_object = SparseCategoricalCrossentropy(from_logits=False, reduction='none')

def loss_function(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), dtype=tf.float32)
    loss = loss_object(y_true, y_pred)
    loss *= mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

optimizer = Adam(learning_rate=1e-4)

# 🔧 Modeli compile et
transformer.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

--------
---------

## MODEL TANIMLAMALARI İÇİN BÜTÜN KODLAR

### 🔧 1️⃣ encoder_block + build_encoder

In [38]:
def encoder_block(x, d_model, num_heads, dropout_rate=0.4):
    attn_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    x = add_and_norm(x, attn_output, dp=dropout_rate)

    ffn = tf.keras.Sequential([
        layers.Dense(d_model * 5, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(d_model)
    ])
    ffn_output = ffn(x)
    x = add_and_norm(x, ffn_output, dp=dropout_rate)

    return x


def build_encoder(vocab_size, max_len, d_model, num_heads, num_layers=3, dropout_rate=0.4):
    encoder_inputs = layers.Input(shape=(None,), name="encoder_input")

    embedding_layer = token_and_position_embedding(vocab_size, max_len, d_model)
    x = embedding_layer(encoder_inputs)

    for _ in range(num_layers):
        x = encoder_block(x, d_model, num_heads, dropout_rate)

    return Model(inputs=encoder_inputs, outputs=x, name="TransformerEncoder")

### 🔧 2️⃣ decoder_block + build_decoder

In [39]:
def decoder_block(x, enc_output, d_model, num_heads, dropout_rate=0.4):
    attn1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x)
    x = add_and_norm(x, attn1, dp=dropout_rate)

    attn2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, enc_output)
    x = add_and_norm(x, attn2, dp=dropout_rate)

    ffn = tf.keras.Sequential([
        layers.Dense(d_model * 5, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(d_model)
    ])
    ffn_output = ffn(x)
    x = add_and_norm(x, ffn_output, dp=dropout_rate)

    return x


def build_decoder(vocab_size, max_len, d_model, num_heads, num_layers=4, dropout_rate=0.4):
    decoder_inputs = layers.Input(shape=(None,), name="decoder_input")
    encoder_outputs = layers.Input(shape=(None, d_model), name="encoder_output")

    embedding_layer = token_and_position_embedding(vocab_size, max_len, d_model)
    x = embedding_layer(decoder_inputs)

    for _ in range(num_layers):
        x = decoder_block(x, encoder_outputs, d_model, num_heads, dropout_rate)

    return Model(inputs=[decoder_inputs, encoder_outputs], outputs=x, name="TransformerDecoder")


### 🧱 3️⃣ build_transformer (Encoder + Decoder Birleşimi)



In [40]:
def build_transformer(input_vocab_size, target_vocab_size, max_len, d_model, num_heads, num_layers=4, dropout_rate=0.4):
    encoder_inputs = layers.Input(shape=(None,), name="encoder_inputs")
    decoder_inputs = layers.Input(shape=(None,), name="decoder_inputs")

    encoder = build_encoder(input_vocab_size, max_len, d_model, num_heads, num_layers, dropout_rate)
    decoder = build_decoder(target_vocab_size, max_len, d_model, num_heads, num_layers, dropout_rate)

    enc_output = encoder(encoder_inputs)
    dec_output = decoder([decoder_inputs, enc_output])

    final_output = layers.Dense(target_vocab_size, activation="softmax", name="output_layer")(dec_output)

    return Model(inputs=[encoder_inputs, decoder_inputs], outputs=final_output, name="TransformerModel")

#### 📌 Örnek Çağırma:

In [44]:
input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

transformer = build_transformer(
    input_vocab_size=input_vocab_size,
    target_vocab_size=target_vocab_size,
    max_len=max_len,
    d_model=256,
    num_heads=4,
    num_layers=6,
    dropout_rate=0.4
)

transformer.summary()


Model: "TransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TransformerEncoder  │ (1, None, 256)    │ 10,286,592 │ encoder_inputs[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TransformerDecoder  │ (1, None, 256)    │ 16,637,696 │ decoder_inputs[0… │
│ (Functional)        │                   │            │ TransformerEncod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (1, None, 251)    │     64,507 │ TransformerDecod… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 26,988,795 (102.95 MB)

 Trainable params: 26,988,795 (102.95 MB)

 Non-trainable params: 0 (0.00 B)

In [45]:
loss_object = SparseCategoricalCrossentropy(from_logits=False, reduction='none')

def loss_function(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), dtype=tf.float32)
    loss = loss_object(y_true, y_pred)
    loss *= mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

optimizer = Adam(learning_rate=1e-4)

# 🔧 Modeli compile et
transformer.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

------
-------

# Şimdi de Eğitim Zamanı

* Model sonucuna takılmayın arkadaşlar.Veri seti 1. aşama olarak 50 veriden oluşuyor ki bu zaten öğrenme konusunda hiçbir fayda sağlamaz.Biz burada temel yapıları çalıştık.Modelin sonucu bizi ilgilendirmiyor.Biz burada işlemleri anlamaya ve teorik olarak oturtmaya çalışıyoruz.İlerleyen zamanlarda projeler yapa yapa sonuca doğru gideceğiz.

In [46]:
transformer.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size=32,
    epochs=50,
    validation_split=0.1  # istersen
)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.0100 - loss: 5.8811 - val_accuracy: 0.0833 - val_loss: 6.6894
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 665ms/step - accuracy: 0.0812 - loss: 5.5551 - val_accuracy: 0.0833 - val_loss: 6.7695
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 688ms/step - accuracy: 0.0833 - loss: 5.4373 - val_accuracy: 0.0833 - val_loss: 5.7509
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 678ms/step - accuracy: 0.0721 - loss: 5.3819 - val_accuracy: 0.0833 - val_loss: 5.5772
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 671ms/step - accuracy: 0.0741 - loss: 5.4143 - val_accuracy: 0.0833 - val_loss: 5.6213
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 662ms/step - accuracy: 0.0779 - loss: 5.3910 - val_accuracy: 0.0833 - val_loss: 5.7628
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 708ms/step - accuracy: 0.0830 - loss: 5.2922 - val_accuracy: 0.0833 - val_loss: 5.7985
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 709ms/step - accuracy: 0.0812 - loss: 5.2403 - val_accuracy: 0.0833 - val_loss: 5

------
------

## Şimdi Modeli Deneyelim

-------
-------

### ✅ 1️⃣ evaluate_sentence() Fonksiyonu

In [47]:
def evaluate_sentence(sentence, tokenizer_in, tokenizer_out, transformer, max_len):
    # 1. Girdiyi tokenize et
    input_seq = tokenizer_in.texts_to_sequences([sentence])
    encoder_input = pad_sequences(input_seq, maxlen=max_len, padding='post')

    # 2. Decoder'ı <start> token ile başlat
    start_token = tokenizer_out.word_index.get('<start>', 1)
    end_token = tokenizer_out.word_index.get('<end>', 2)

    decoder_input = [start_token]
    result = []

    for _ in range(max_len):
        # Padle
        dec_in = pad_sequences([decoder_input], maxlen=max_len, padding='post')

        # Model tahmini
        predictions = transformer.predict([encoder_input, dec_in], verbose=0)
        predicted_id = tf.argmax(predictions[0, len(decoder_input)-1]).numpy()

        # <end> geldiyse dur
        if predicted_id == end_token:
            break

        result.append(predicted_id)
        decoder_input.append(predicted_id)

    # ID → kelime çevir
    reverse_target_word_index = {i: w for w, i in tokenizer_out.word_index.items()}
    predicted_sentence = ' '.join([reverse_target_word_index.get(i, '?') for i in result])

    return predicted_sentence


#### 📌 Kullanımı:

In [49]:
test_input = "Merhaba"
output = evaluate_sentence(test_input, input_tokenizer, target_tokenizer, transformer, max_len=max_len)
print("Bot:", output)

Bot: bir bir bir bir


------
-------

# 🧠 MASKING – Transformer’da Ne İşe Yarar?

#### Transformer paralel çalıştığı için:

* Tüm cümle bir anda işlenir

Ama bazı yerleri gizlemek gerekir:

* PAD tokenları (eğitimi bozmasın diye)

* Decoder'da gelecekteki kelimeler (daha yazılmamışsa bakmamalı)

İşte bu durumda devreye "masking" girer ✅

## 🎭 MASKING TÜRLERİ
#### 1️⃣ Padding Mask (Gerçek cümle ≠ pad token)

* PAD token'lar model için bilgi taşımaz

* Hem encoder hem decoder'da kullanılır

Cümle:      merhaba nasılsın <pad <pad

Mask:       1          1         0     0


### 2️⃣ Look-Ahead Mask (Causal Masking)
* Decoder kendinden sonraki kelimeye bakamaz

* Çünkü henüz yazılmamıştır

* Yani decoder_input[t] → sadece [:t]’ye kadar bakar

Örn (3 kelime için):

1 0 0

1 1 0

1 1 1

* Bu bir üst üçgen matris'tir

### 3️⃣ Combined Mask
* Padding + look-ahead birlikte

* Decoder’ın hem pad’leri hem geleceği görmemesini sağlar

## 🔧 Nerelerde Kullanılır?
| Maske Türü     | Kullanıldığı Yer             |
|----------------|-------------------------------|
| Padding Mask   | Encoder & Decoder Attention   |
| Look-Ahead     | Decoder Self-Attention        |
| Combined Mask  | Decoder’ın giriş kısmında     |

### 💡 Neden Bu Kadar Önemli?
❌ Eğer mask yapılmazsa:

* Pad token’lar sıfır olduğu halde öğrenme hatası oluşur

* Decoder ilerideki kelimeyi görerek “hile yapar”

✅ Mask → Doğru öğrenmeyi garanti eder


### Ama asıl kritik soru şu:
⚙️ "Transformer’ın neresine bu mask’leri enjekte edeceğiz?"

## 🧠 Özet: Masking'i Nerede Kullanıyoruz?

| Nerede?                       | Ne Veriyoruz?                   |
|-------------------------------|----------------------------------|
| 🔹 Encoder Self-Attention     | `attention_mask = padding_mask` |
| 🔹 Decoder Self-Attention     | `attention_mask = combined_mask`|
| 🔹 Decoder Enc-Dec Attention  | `attention_mask = padding_mask` |


### Şimdi kodlamaya geçelim.Ama unutmayın.Biz bu mask işlemlerinin hepsini uyguluyoruz.Yalnızca yukarıda duran kavramlara göre yapacağız.Yani farklı katmanlara enjekte edeceğiz.

## 🔹 1️⃣ create_padding_mask() – TEORİ
Ne yapar?

* Giriş tensöründe (örneğin [5, 7, 0, 0])

* 0 olanları tespit eder (bunlar <pad>)

* Bunları 1 yapar → yani maskelenecek yer

* Ardından True → 1.0, False → 0.0 olarak float mask üretir

In [58]:
def create_padding_mask(seq):
    # PAD token = 0 → True olur
    mask = tf.cast(tf.math.equal(seq, 0), tf.float32)

    # [batch, seq_len] → [batch, 1, 1, seq_len]
    # Bu, MultiHeadAttention için beklenen shape’tir
    return mask[:, tf.newaxis, tf.newaxis, :]


#### 📌 Örnek:

In [53]:
seq = tf.constant([[7, 6, 0, 0, 0]])
mask = create_padding_mask(seq)
print(mask)

tf.Tensor([[[[0. 0. 1. 1. 1.]]]], shape=(1, 1, 1, 5), dtype=float32)


#### 🔍 Çıktı:


[[[[0. 0. 1. 1. 1.]]]]

* Yani 0 olan yerlere dikkat edilmesin dedik ✅

## 🔹 2️⃣ create_look_ahead_mask() – TEORİ
Amaç:

* Decoder kendi cümlesini üretirken gelecekteki kelimelere bakmamalı

* Yani t=3 anındayken sadece 0, 1, 2 pozisyonlarını görmeli

* Bu, causal (nedensel) attention sağlar → kelime sırasına sadık kalınır ✅

In [59]:
def create_look_ahead_mask(size):
    # tf.ones((size, size)) → [size x size] boyutunda sadece 1'lerden oluşan matris üretir
    # Örn: size=4 ise → [[1 1 1 1], [1 1 1 1], [1 1 1 1], [1 1 1 1]]

    # tf.linalg.band_part(tensor, num_lower, num_upper):
    # - 'band_part' fonksiyonu matrisi üçgen hale getirir
    # - num_lower=-1 → alttaki tüm satırları tut
    # - num_upper=0  → sadece diagonal ve alt taraf kalır, üst kısmı sıfırlar

    # tf.linalg.band_part(tf.ones(...), -1, 0)
    # → Alt üçgen matris üretir:
    # [[1 0 0 0]
    #  [1 1 0 0]
    #  [1 1 1 0]
    #  [1 1 1 1]]

    # 1 - (...) → üst üçgeni 1 yapar:
    # [[0 1 1 1]
    #  [0 0 1 1]
    #  [0 0 0 1]
    #  [0 0 0 0]]

    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)

    return mask  # shape: [size, size]


### 📌 Örnek:

In [56]:
mask = create_look_ahead_mask(5)
print(mask)

tf.Tensor(
[[0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1.]
 [0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]], shape=(5, 5), dtype=float32)


* Yani pozisyon t yalnızca 0:t aralığını görebilir.

* Yukarısı 1 → gizlenecek

## 🔗 Combined Mask = Padding Mask + Look-Ahead Mask

* Bu maskeyi decoder’ın self-attention kısmında kullanıyoruz.

Amaç:

🔒 Geleceğe bakamasın (look-ahead)

🧼 Pad token'lara dikkat etmesin (padding)

### 🧠 Neden Combine Ediyoruz?
* Tek başına look_ahead_mask:

Sadece geleceği gizler, ama pad’leri görür ❌

* Tek başına padding_mask:

Pad’leri gizler, ama geleceğe bakar ❌

* Bunları tf.maximum() ile birleştirince:

Neresi 1 ise orayı gizle → hem pad hem gelecek ✔️

In [61]:
def create_combined_mask(decoder_input):

    look_ahead = create_look_ahead_mask(tf.shape(decoder_input)[1])

    padding = create_padding_mask(decoder_input)

    look_ahead = tf.maximum(look_ahead,padding[:,:,0:])

    return look_ahead[:,tf.newaxis,:,:]

### Yukarıda bulunan kodlarla temel masking bilgisini anlatmaya çalıştım.Şimdi bunları nasıl modele aktaracağımıza bakalım.

----
---

## Hazırladığımız padding_mask, look_ahead_mask, combined_mask fonksiyonlarınışimdi encoder_block ve decoder_block'a entegre ediyoruz.

### 🧱 1️⃣ encoder_block – Mask Eklemesi

In [73]:
def encoder_block(x, d_model, num_heads, dropout_rate, padding_mask=None):
    attn_output = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=d_model
    )(x, x, attention_mask=padding_mask)  # Maske burada kullanılıyor

    x = add_and_norm(x, attn_output, dp=dropout_rate)

    ffn = tf.keras.Sequential([
        Dense(d_model * 4, activation='relu'),
        Dropout(dropout_rate),
        Dense(d_model)
    ])
    ffn_output = ffn(x)
    x = add_and_norm(x, ffn_output, dp=dropout_rate)

    return x


### 🔧 1️⃣ build_encoder() – Mask Entegreli

In [75]:
def build_encoder(vocab_size, max_len, d_model, num_heads, num_layers=1, dropout_rate=0.1):
    encoder_inputs = Input(shape=(None,), name="encoder_input")

    embedding_layer = token_and_position_embedding(vocab_size, max_len, d_model)
    x = embedding_layer(encoder_inputs)

    # Padding mask (mask shape: [batch, 1, 1, seq_len])
    padding_mask = Lambda(lambda seq: create_padding_mask(seq))(encoder_inputs)

    for _ in range(num_layers):
        x = encoder_block(x, d_model, num_heads, dropout_rate, padding_mask=padding_mask)

    return Model(inputs=encoder_inputs, outputs=x, name="TransformerEncoder")


### 🔁 2️⃣ decoder_block – 2 Maske Eklemesi

In [76]:
def decoder_block(x, enc_output, d_model, num_heads,
                  dropout_rate=0.1, combined_mask=None, padding_mask=None):
    # 1. Decoder Self-Attention (geleceği görememeli)
    attn1 = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=d_model
    )(x, x, attention_mask=combined_mask)  # ✅ Look-Ahead + Padding mask

    x = add_and_norm(x, attn1, dp=dropout_rate)

    # 2. Encoder-Decoder Attention (pad'leri gizle)
    attn2 = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=d_model
    )(x, enc_output, attention_mask=padding_mask)  # ✅ Padding mask

    x = add_and_norm(x, attn2, dp=dropout_rate)

    # 3. Feed-Forward
    ffn = tf.keras.Sequential([
        Dense(d_model * 4, activation='relu'),
        Dropout(dropout_rate),
        Dense(d_model)
    ])
    ffn_output = ffn(x)
    x = add_and_norm(x, ffn_output, dp=dropout_rate)

    return x


### 🔁 2️⃣ build_decoder() – Full Mask Entegreli

In [77]:
def build_decoder(vocab_size, max_len, d_model, num_heads, num_layers=1, dropout_rate=0.1):
    decoder_inputs = Input(shape=(None,), name="decoder_input")
    encoder_outputs = Input(shape=(None, d_model), name="encoder_output")

    embedding_layer = token_and_position_embedding(vocab_size, max_len, d_model)
    x = embedding_layer(decoder_inputs)

    # Maskleri oluştur
    padding_mask = Lambda(lambda seq: create_padding_mask(seq))(decoder_inputs)
    look_ahead = Lambda(lambda seq: create_look_ahead_mask(tf.shape(seq)[1]))(decoder_inputs)
    combined_mask = Lambda(lambda args: tf.maximum(args[0], args[1]))([look_ahead, padding_mask])

    for _ in range(num_layers):
        x = decoder_block(
            x,
            encoder_outputs,
            d_model,
            num_heads,
            dropout_rate,
            combined_mask=combined_mask,
            padding_mask=padding_mask
        )

    return Model(inputs=[decoder_inputs, encoder_outputs], outputs=x, name="TransformerDecoder")


### 🧱 build_transformer() – Mask Entegreli Final Sürüm

In [78]:
def build_transformer(input_vocab_size, target_vocab_size, max_len,
                      d_model, num_heads, num_layers=1, dropout_rate=0.1):
    
    # Girişler
    encoder_inputs = Input(shape=(None,), name="encoder_inputs")
    decoder_inputs = Input(shape=(None,), name="decoder_inputs")

    # Encoder ve Decoder modellerini oluştur
    encoder = build_encoder(input_vocab_size, max_len, d_model, num_heads, num_layers, dropout_rate)
    decoder = build_decoder(target_vocab_size, max_len, d_model, num_heads, num_layers, dropout_rate)

    # Encoder output
    enc_output = encoder(encoder_inputs)

    # Decoder output (encoder output'u da alır)
    dec_output = decoder([decoder_inputs, enc_output])

    # Final Dense layer: her timestep'te vocab boyutunda softmax
    final_output = Dense(target_vocab_size, activation='softmax', name="output_layer")(dec_output)

    # Modeli tanımla
    return Model(inputs=[encoder_inputs, decoder_inputs], outputs=final_output, name="TransformerModel")


In [80]:
transformer = build_transformer(
    input_vocab_size=input_vocab_size,
    target_vocab_size=target_vocab_size,
    max_len=max_len,
    d_model=256,
    num_heads=4,
    num_layers=3,
    dropout_rate=0.5
)
transformer.summary()


Model: "TransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_inputs      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TransformerEncoder  │ (1, None, 256)    │  4,763,136 │ encoder_inputs[0… │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ TransformerDecoder  │ (1, None, 256)    │  7,956,992 │ decoder_inputs[0… │
│ (Functional)        │                   │            │ TransformerEncod… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (1, None, 251)    │     64,507 │ TransformerDecod… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,784,635 (48.77 MB)

 Trainable params: 12,784,635 (48.77 MB)

 Non-trainable params: 0 (0.00 B)

### ✅ 1️⃣ Loss Fonksiyonu – PAD maskeli

In [81]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    reduction='none'  # PAD için maskeleme uygulayacağız
)

def loss_function(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)  # PAD'ler 0'dır → onları dışla
    loss = loss_object(y_true, y_pred)
    loss *= mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)



### ⚙️ 2️⃣ Optimizer - Compile Et

In [83]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
transformer.compile(loss=loss_function, optimizer=optimizer, metrics=["accuracy"])

### 📦 4️⃣ Eğitime Başla (fit)

In [84]:
transformer.fit(
    [encoder_input, decoder_input],
    decoder_target,
    batch_size=32,
    epochs=20,
    validation_split=0.1  # istersen ayır
)


Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.0012 - loss: 5.9597 - val_accuracy: 0.0833 - val_loss: 5.6882
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 379ms/step - accuracy: 0.0339 - loss: 5.7190 - val_accuracy: 0.0833 - val_loss: 6.4323
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 346ms/step - accuracy: 0.0728 - loss: 5.5923 - val_accuracy: 0.0833 - val_loss: 7.1825
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 364ms/step - accuracy: 0.0833 - loss: 5.4467 - val_accuracy: 0.0833 - val_loss: 7.2245
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 341ms/step - accuracy: 0.0846 - loss: 5.4262 - val_accuracy: 0.0833 - val_loss: 6.7729
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 352ms/step - accuracy: 0.0800 - loss: 5.3585 - val_accuracy: 0.0833 - val_loss: 6.3168
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 352ms/step - accuracy: 0.0842 - loss: 5.3457 - val_accuracy: 0.1167 - val_loss: 6.0718
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 361ms/step - accuracy: 0.0749 - loss: 5.2766 - val_accuracy: 0.1500 - val_loss: 5

-------
-----
-----

# Şimdi bütün bu öğrendiğimiz terimleri bir koda geçireceğiz.Temel yapı taşlarını oturttuk şimdi önemli olan bu kodları iyice teorikle beraber anlamak.Burda bahsedilmek istenilen kod parçalarına (( transformer_1_kodları.ipynb ))  içerisinden ulaşabilirsiniz.Karmaşıklığı engellemek için oradan inceleyiniz.

-----
--------
------